In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", token=hf_token)

/home/junseokkim00/anaconda3/envs/biasdebate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.04s/it]


In [2]:
device='cuda:0'
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [3]:
!nvidia-smi

Mon Mar 31 22:37:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:18:00.0 Off |                    0 |
| 30%   38C    P2              61W / 230W |  12466MiB / 23028MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
feedback_prompt="Review your previous answer and find problems with your answer."
refine_prompt="Based on the problems you found, improve your answer. Please reiterate your answer, with your final answer a single numerical number, in the form \boxed{answer}."
answer_trigger_prompt='Therefore, the answer (arabic numerals) is'

In [5]:
question="Five friends eat at a fast-food chain and order the following: 5 pieces of hamburger that cost $3 each; 4 sets of French fries that cost $1.20; 5 cups of soda that cost $0.5 each; and 1 platter of spaghetti that cost $2.7. How much will each of them pay if they will split the bill equally?"

In [6]:
chat=[
    {"role": "user", "content": question}
]

In [7]:
def inference(chat, model, tokenizer, device):
    tokenized_chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    tokenized_chat = tokenized_chat.to(device)
    output = model.generate(tokenized_chat, max_new_tokens=1024, do_sample=True)
    # output.shape
    decode_input = tokenizer.batch_decode(tokenized_chat, skip_special_tokens=True)[0]
    decode_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return decode_output[len(decode_input):]
    

In [8]:
initial_response = inference(chat, model, tokenizer, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [9]:
initial_response

'To find the total cost, we need to calculate the cost of each item and add them together. \n\n- 5 pieces of hamburger: 5 * $3 = $15\n- 4 sets of French fries: 4 * $1.20 = $4.80\n- 5 cups of soda: 5 * $0.5 = $2.50\n- 1 platter of spaghetti: $2.7\n\nTotal cost: $15 + $4.80 + $2.50 + $2.7 = $25.00\n\nSince there are 5 friends, they will split the bill equally. \n\n$25.00 / 5 = $5.00\n\nEach friend will pay $5.00.'

In [10]:
chat.append({"role": "assistant", "content": initial_response})

In [11]:
chat.append({"role": "user", "content": feedback_prompt})
feedback = inference(chat, model, tokenizer, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
feedback

"Upon reviewing my previous answer, I noticed that I made a calculation error in the total cost. Here's the correct calculation:\n\n- 5 pieces of hamburger: 5 * $3 = $15\n- 4 sets of French fries: 4 * $1.20 = $4.80\n- 5 cups of soda: 5 * $0.5 = $2.50\n- 1 platter of spaghetti: $2.7\n\nTotal cost: $15 + $4.80 = $19.80\n$19.80 + $2.50 = $22.30\n$22.30 + $2.7 = $25.00\n\nThe correct total cost is indeed $25.00.\n\nHowever, I also noticed that I made a mistake in the final calculation. \n\n$25.00 / 5 = $5.00\n\nThis is correct, but I can provide a more detailed explanation.\n\nTo split the bill equally, we can divide the total cost by the number of friends. Since there are 5 friends, each friend will pay:\n\n$25.00 ÷ 5 = $5.00\n\nThis means that each friend will pay $5.00."

In [13]:
chat.append({"role": "assistant", "content": feedback})

In [14]:
chat.append({"role": "user", "content": refine_prompt})
revision = inference(chat, model, tokenizer, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [15]:
revision

'To find the total cost, we need to calculate the cost of each item and add them together. \n\n- 5 pieces of hamburger: 5 * $3 = $15\n- 4 sets of French fries: 4 * $1.20 = $4.80\n- 5 cups of soda: 5 * $0.5 = $2.50\n- 1 platter of spaghetti: $2.7\n\nTotal cost: \n1. $15 + $4.80 = $19.80\n2. $19.80 + $2.50 = $22.30\n3. $22.30 + $2.7 = $25.00\n\nSince there are 5 friends, they will split the bill equally. \n\n$25.00 / 5 = $5.00\n\nEach friend will pay $5.00.\n\n**$5.00**'

In [16]:
chat

[{'role': 'user',
  'content': 'Five friends eat at a fast-food chain and order the following: 5 pieces of hamburger that cost $3 each; 4 sets of French fries that cost $1.20; 5 cups of soda that cost $0.5 each; and 1 platter of spaghetti that cost $2.7. How much will each of them pay if they will split the bill equally?'},
 {'role': 'assistant',
  'content': 'To find the total cost, we need to calculate the cost of each item and add them together. \n\n- 5 pieces of hamburger: 5 * $3 = $15\n- 4 sets of French fries: 4 * $1.20 = $4.80\n- 5 cups of soda: 5 * $0.5 = $2.50\n- 1 platter of spaghetti: $2.7\n\nTotal cost: $15 + $4.80 + $2.50 + $2.7 = $25.00\n\nSince there are 5 friends, they will split the bill equally. \n\n$25.00 / 5 = $5.00\n\nEach friend will pay $5.00.'},
 {'role': 'user',
  'content': 'Review your previous answer and find problems with your answer.'},
 {'role': 'assistant',
  'content': "Upon reviewing my previous answer, I noticed that I made a calculation error in the 

In [17]:
chat.append({'role': 'assistant', 'contet': revision})

In [18]:
extract_answer={'role': 'user', 'content': 'Therefore, the answer (arabic numerals) is'}
final_answer = inference(chat+[extract_answer], model, tokenizer, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [19]:
final_answer

'$5.00'

## Now let's make it to generate $k$ feedback and $n$ revisions

In [40]:
import re
def check(pred, label):
    try:    
        pred = pred.replace(',','')
        label = label.replace(',','')
        pred = re.findall(r"[-+]?(?:\d*\.*\d+)",pred)[0]
        if pred == label:
            return True
    except:
        pass
    return False

In [39]:
pred="{5}"
label="5"
pred = re.findall(r"[-+]?(?:\d*\.*\d+)",pred)
pred

['5']

In [33]:
def generate_data(model, tokenizer, question, answer, k, n, device):
    insts=[]
    chat=[
        {"role": "user", "content": question}
    ]
    initial_response = inference(chat, model, tokenizer, device)
    print("initial_response: ", initial_response)
    chat.append({"role": "assistant", "content": initial_response})
    chat.append({"role": "user", "content": feedback_prompt})
    # TODO: need to save verbose
    for idx in range(k):
        feedback = inference(chat, model, tokenizer, device)
        partial_chat=[
            {"role": "assistant", "content": feedback},
            {"role": "user", "content": refine_prompt}
        ]
        cnt=0
        print(f"feedback #{idx}: ", feedback)
        for _ in range(n):
            revision = inference(chat+partial_chat, model, tokenizer, device)
            print(revision)
            extract_answer_chat =[
                {"role": "assistant", "content": revision},
                {"role": "user", "content": answer_trigger_prompt}
            ]
            pred = inference(chat+partial_chat+extract_answer_chat, model, tokenizer, device)
            print(pred)
            if check(pred, answer):
                cnt+=1
        print(f"score for feedback #{idx}: ", cnt/n*100)
        insts.append({
            'question': question,
            'initial_response': initial_response,
            'feedback': feedback,
            'score': cnt / n * 100
        })
    return insts

In [34]:
insts = generate_data(model, tokenizer, question, "5", 3, 5, device)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


initial_response:  To find the total cost, we need to calculate the cost of each item and add them together. 

The cost of the hamburgers: 5 pieces * $3 each = $15
The cost of the French fries: 4 sets * $1.20 each = $4.80
The cost of the sodas: 5 cups * $0.5 each = $2.50
The cost of the spaghetti: $2.70

Total cost: $15 + $4.80 + $2.50 + $2.70 = $25

Since there are 5 friends, we will split the total cost equally among them. 

$25 / 5 = $5

Each friend will pay $5.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


feedback #0:  Upon reviewing my previous answer, I noticed a potential issue. 

When calculating the cost of the French fries, I should have multiplied the cost per set by the number of sets. However, I only multiplied the cost per set by the number of sets, but then I forgot to multiply it by the cost per set again, so the correct calculation is 4 sets * $1.20 * 5 = $24. 

Here's the corrected calculation:

The cost of the hamburgers: 5 pieces * $3 each = $15
The cost of the French fries: 4 sets * $1.20 * 5 = $24
The cost of the sodas: 5 cups * $0.5 each = $2.50
The cost of the spaghetti: $2.70

Total cost: $15 + $24 + $2.50 + $2.70 = $44.20

Since there are 5 friends, we will split the total cost equally among them. 

$44.20 / 5 = $8.84

Each friend will pay $8.84.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

To find the total cost, we need to calculate the cost of each item and add them together. 

The cost of the hamburgers: 5 pieces * $3 each = $15
The cost of the French fries: 4 sets * $1.20 each = $4.80
The cost of the sodas: 5 cups * $0.5 each = $2.50
The cost of the spaghetti: $2.70

Total cost: $15 + $4.80 + $2.50 + $2.70 = $25

Since there are 5 friends, we will split the total cost equally among them. 

$25 / 5 = $5

Each friend will pay $5.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


boxed{5}


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


To find the total cost, we need to calculate the cost of each item and add them together. 

The cost of the hamburgers: 5 pieces * $3 each = $15
The cost of the French fries: 4 sets * $1.20 each = $4.80
The cost of the sodas: 5 cups * $0.5 each = $2.50
The cost of the spaghetti: $2.70

Total cost: $15 + $4.80 + $2.50 + $2.70 = $25

Since there are 5 friends, we will split the total cost equally among them. 

$25 / 5 = $5

Each friend will pay $5.
boxed{5}
score for feedback #2:  100.0
